In [ ]:
%run ../load_files.ipynb

In [ ]:
%run ../data_dti_matrixes.ipynb

In [ ]:
%run ../y_multilevel.ipynb

In [ ]:
y = y_values.memory

In [ ]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, stratify=y)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', LogisticRegression(solver='liblinear'))
])

In [ ]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 50, 3)
PENALTY = ['l1', 'l2']
REGULATION_C = np.linspace(1, 100, 90)
MATRIXES = [['DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', ], ['DTI_FA']]
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            


param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__penalty': PENALTY,
        'classify__C' : REGULATION_C,
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [ ]:
grid.fit(X_train, y_train)

In [ ]:
%run ../grid_results.ipynb